In [231]:
###config에서 바로 성능 예측 (tps, latency by MLP)
import os
import utils
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import davies_bouldin_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import configparser
os.chdir("/home/sein/ksc_model/model")

def mysql_knob_dataframe(wk, knobs_path):
    knobs_path = os.path.join(knobs_path, str(wk))
    config_len = len(os.listdir(knobs_path))
    cnf_parser = configparser.ConfigParser()
    pd_mysql = pd.DataFrame()
    for idx in range(config_len):
        cnf_parser.read(os.path.join(knobs_path, f'my_{idx}.cnf'))
        conf_dict = cnf_parser._sections['mysqld']
        tmp = pd.DataFrame(data=[conf_dict.values()], columns=conf_dict.keys())
        pd_mysql = pd.concat([pd_mysql, tmp])
        
    pd_mysql = pd_mysql.reset_index(drop=True)
    pd_mysql = pd_mysql.drop(columns=['log-error', 'bind-address'])
    return pd_mysql

def get_data(knob_path, external_path, wk):
    raw_knobs = mysql_knob_dataframe(wk, knob_path)
    external_ = pd.read_csv(os.path.join(external_path, f'external_results_{wk}.csv'), index_col=0)
    latency_columns = []
    for col in external_.columns:
        if col.find("latency") == 0 and col != 'latency_max' and col != 'latency_CLEANUP':
            latency_columns.append(col)
    external = external_[['tps']].copy()
    external['latency'] = external_[latency_columns].max(axis=1)
    return raw_knobs, external



In [22]:
os.chdir("/home/sein/ksc_model/model")

In [23]:
import torch
import torch.nn as nn


class Pred_Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Pred_Model, self).__init__()
        # self.knob_fc = nn.Sequential(nn.Linear(self.input_dim, self.hidden_dim), nn.ReLU())  #hidden layer
        # self.im_fc = nn.Sequential(nn.Linear(self.hidden_dim, self.output_dim), nn.Sigmoid()) #output layer
    # dropout = nn.Dropout(p=0.25)
        #--------------------------------------
        self.input_dim = input_dim # 22
        self.hidden_dim = hidden_dim 
        self.hidden_dim2 = hidden_dim//2
        # self.hidden_dim3 = hidden_dim//4
        # self.hidden_dim4 = hidden_dim//2
        self.output_dim = output_dim #3
       #--------------------------------------
       
       
        # self.weight = nn.parameter (torch.FloatTensor(7,32,32, device="cuda"))
        
        
        self.Layer1 = nn.Sequential(nn.Linear(self.input_dim, self.hidden_dim), nn.GELU())  #hidden layer
        self.Layer2 = nn.Sequential(nn.Linear(self.hidden_dim, self.hidden_dim2), nn.GELU())
        # self.Layer3 = nn.Sequential(nn.Linear(self.hidden_dim2, self.hidden_dim3), nn.ReLU())
        # self.Layer4 = nn.Sequential(nn.Linear(self.hidden_dim3, self.hidden_dim4), nn.ReLU())
        self.Layer3 = nn.Sequential(nn.Linear(self.hidden_dim2, self.output_dim))
        
        #self.Layer4 = nn.Sequential(nn.LayerNorm(self.hidden_dim3),nn.Linear(self.hidden_dim3, self.output_dim), nn.Softmax(dim=1))

    def forward(self, x):
       
        x = self.Layer1(x)
        x = self.Layer2(x)
        x = self.Layer3(x)
        # x = self.Layer4(x)
        # x = self.Layer5(x)

        # self.x_kb = self.knob_fc(x)
        # self.
        # self.x_im = self.im_fc(self.x_kb)

      
        return x


In [232]:

### conf - performance prediction model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
import pandas as pd
from sklearn.metrics import mean_squared_error

random_seed = 777
import random
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)
#workload6에 대한 external metric
KNOB_PATH = ('../data/configs_new_dataset/')
EXTERNAL_PATH = ("../data/configs_new_dataset/external_new_dataset/")
wk = 6

# print(EXTERNAL)

raw_knobs, external = get_data(KNOB_PATH, EXTERNAL_PATH, wk)
# print(raw_knobs)
config_Preprocessing = []
raw_knobsT = raw_knobs.transpose()
# print(raw_knobsT) 
for i in range(20):
    chunk_config = raw_knobsT.iloc[i]
    # print(chunk_config)
    chunk_config = chunk_config.values.reshape(-1, 1) # (2000,1)
    # print(chunk_config)
    # chunk_config = raw_knobs.iloc[:,[i]]
    scaler_config = StandardScaler().fit(chunk_config)
    # print(scaler_config)
    Preprocessing = scaler_config.transform(chunk_config)
    Preprocessing = pd.DataFrame(Preprocessing) # (2000,1)
    # print(Preprocessing)
    config_Preprocessing.append(Preprocessing)
    # print(config_Preprocessing)
config_Preprocessing = pd.concat(config_Preprocessing, axis=1)
#print(config_Preprocessing.shape)



    
#     config_Preprocessing.append(Preprocessing)
#     config_Preprocessing = pd.DataFrame(config_Preprocessing)
#     #여기까지 
# print(config_Preprocessing)
# config_Preprocessing = pd.concat([Preprocessing], axis = 1)
    

    # config_Preprocessing 
    # print(config_Preprocessing)  
# config_Preprocessing = pd.concat([Preprocessing], axis = 1)
      
# config_Preprocessing.append(Preprocessing)
# print(config_Preprocessing)

    # config_Preprocessing = pd.concat([Preprocessing], axis = 1)



# print(external)
#raw_knobs 대한 scaling
# scaler_config = MinMaxScaler().fit(raw_knobs)
# config_Preprocessing = scaler_config.transform(raw_knobs)
# config_Preprocessing = pd.DataFrame(config_Preprocessing)


# scaler_metrics = Normalizer().fit(external)
# metrics_Preprocessing = scaler_metrics.transform(external)
# metrics_Preprocessing = pd.DataFrame(metrics_Preprocessing)

tps = external['tps']
latency = external['latency']
tps = tps.values.reshape(-1,1)
latency = latency.values.reshape(-1,1)
scaled_tps = MinMaxScaler().fit(tps)
# scaled_tps = scaled_tps.reshape(-1, 1)
tps_preprocessing = scaled_tps.transform(tps)
tps_preprocessing = pd.DataFrame(tps_preprocessing)

scaled_latency = MinMaxScaler().fit(latency)
# scaled_latency = scaled_latency.reshape(-1, 1)
latency_preprocessing = scaled_tps.transform(latency)
latency_preprocessing = pd.DataFrame(latency_preprocessing)

external_preprocessing = pd.concat([tps_preprocessing, latency_preprocessing], axis =1)



# scaled_external = StandardScaler().fit(external)
# external_preprocessing = scaled_external.transform(external)
# external_preprocessing = pd.DataFrame(external_preprocessing)

# np_external = external.to_numpy()
# scaler_metrics = MinMaxScaler().fit(np_external)
# metrics_Preprocessing = scaler_metrics.transform(np_external)
# metrics_Preprocessing = pd.DataFrame(metrics_Preprocessing)
# metrics_Preprocessing = metrics_Preprocessing.astype('float')
# print(type(config_Preprocessing))
# print(type(metrics_Preprocessing))
#X = configuraion, Y = scaling external metrics
X_train, X_test, y_train, y_test = train_test_split(config_Preprocessing, external_preprocessing, test_size=0.4, shuffle=True)


# train_index = X_train.index
# test_index = X_test.index

#X,Y값 scaling
# X_scaler = StandardScaler().fit(X_train)
# Y_scaler = StandardScaler().fit(y_train)

scaled_X_train = X_train
scaled_X_test = X_test
scaled_Y_train = y_train
scaled_Y_test = y_test



# scaled_X_train = X_scaler.transform(X_train)
# scaled_X_test = X_scaler.transform(X_test)
# scaled_Y_train = Y_scaler.transform(y_train)
# scaled_Y_test = Y_scaler.transform(y_test)
# train_TPS = y_train[:,0]
# train_Latency = y_train[:,1]
# test_TPS = y_test[:,0]
# test_Latency = y_test[:,1]
import torch
# scaled_X_train, scaled_Y_train = scaled_X_train.values, scaled_Y_train.values
# scaled_X_test, scaled_Y_test = scaled_X_test.values, scaled_Y_test.values
scaled_X_train, scaled_Y_train = torch.Tensor(scaled_X_train.values), torch.Tensor(scaled_Y_train.values).cuda()
scaled_X_test, scaled_Y_test = torch.Tensor(scaled_X_test.values), torch.Tensor(scaled_Y_test.values).cuda()


# print(scaled_X_train.type())
from torch.utils.data import TensorDataset, DataLoader
dataset_tr = TensorDataset(scaled_X_train, scaled_Y_train)
dataset_te = TensorDataset(scaled_X_test, scaled_Y_test)
# dataset_tr = (scaled_X_train, scaled_Y_train)
# dataset_te = (scaled_X_test, scaled_Y_test)
## prepare experiment
batch_size = 128
dataloader_tr = DataLoader(dataset_tr, batch_size=batch_size, shuffle=True)
dataloader_te = DataLoader(dataset_te, batch_size=batch_size, shuffle=False)

import torch.optim as optim
from sklearn.metrics import r2_score
#Attention
# network = model(input_dim = len(scaled_X_train[0]), hidden_dim = 32, output_dim = 2).cuda()
# network = ReshapeNet(input_dim = len(scaled_X_train[0]), hidden_dim = 32, output_dim = 2, group_dim=8).cuda()
criterion = nn.MSELoss().cuda() #regression
# loss = F.mse_loss(output, target)
####추후 수정
# import warnings
# warnings.filterwarnings('ignore')
network = Pred_Model(input_dim = len(scaled_X_train[0]), hidden_dim = 64, output_dim = 2).cuda()
    #train
# optimizer = optim.Adam(network.parameters(), lr=0.0001, weight_decay=0.001)
optimizer = optim.Adam(network.parameters(), lr=0.0001, weight_decay=0.001)

# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# def get_lr(optimizer):
#     for param_group in optimizer.param_groups:
#         return param_group['lr']

for epoch in range(800):
    #loss_tr = self.train(self.model, dataloader_tr)
    total_loss = 0.
    # total_acc = 0.
    ## start iteration
    network.train()
    # for parameter in network.parameters():
    #     print(parameter.requires_grad) 
    for data, target in dataloader_tr:
        optimizer.zero_grad()
        
        data = np.array(data)
        # onehot = clf.predict(data) #one-hot classification model의 결과
        
        # stack_onehot = []
        
        # for i in onehot:
        #    onehot_li = [0,0,0]
        #    onehot_li[i] += 1 
        #    stack_onehot.append(onehot_li)
        # stack_onehot = torch.tensor(stack_onehot)
       
        # #stack_onehot = torch.stack(torch.tensor(stack_onehot))
        # #print(stack_onehot.shape)
        # cluster = stack_onehot.float().cuda()
        
        data = torch.tensor(data).float().cuda()
        
        #x,y 를 받음 / conf-linear-attention & onehot-linear-attention
        metrics = network(data) #metrics = 
        # metrics = metrics.float()
        target = target.float().cuda()
        
    # print(metrics)    ###metric's shape = [tps, latency]
    # print(metrics.shape)   
    # print(target)     ###target's shape = [tps, latency]
    # print(target.shape)
    
        loss = criterion(metrics, target)
        #mse = mean_squared_error(target.detach().cpu().numpy(), metrics.detach().cpu().numpy())
        # mse = mse.cpu().numpy()
        #loss = np.sqrt(mse)
        
       
        #backpropagation
        loss.backward()
        # update the model's weight(parameter value)
        optimizer.step()

        total_loss += loss.item() 
        
        # print(get_lr(optimizer))
        
        # total_acc = accuracy_score(target,metrics)
        # total_acc = total_acc.cpu().numpy()
        # correct_pred = metrics == target
        # accuracy = correct_pred.float().mean()
        # total_acc += accuracy 
    ## calculte mean of loss 
    total_loss /= len(dataloader_tr)
    # total_acc /= len(dataloader_tr)
    # print(f'[{epoch+1}epoch] tr_loss:{total_loss}')
    
    ## check a log (real-time experiment performance)
    if epoch % 10 == 0:
        total_val_loss = 0
        # total_val_acc = 0
        
        ## validation performance check
        network.eval()
        with torch.no_grad():
            # trues = []
            # preds = []
            preds = torch.Tensor().cuda()
            trues = torch.Tensor().cuda()
    
            for data, target in dataloader_te:
                data = np.array(data)
                # onehot = clf.predict(data)
                # stack_onehot = []
                
                # for i in onehot:
                #     onehot_li = [0,0,0]
                #     onehot_li[i] += 1 
                #     stack_onehot.append(onehot_li)
                # stack_onehot = torch.tensor(stack_onehot)

                # cluster = stack_onehot.float().cuda()                


                data = torch.tensor(data).float().cuda()
                metrics = network(data)
                #metrics = linear-attention-linear
                metrics = metrics.float().cuda()
             
                
                target = target.float().cuda()
                # mse = mean_squared_error(target, metrics)
                # loss = np.sqrt(mse)
                loss = criterion(metrics, target)  
                total_val_loss += loss.item()
                
                # correct_pred = metrics == target
                # accuracy = correct_pred.float().mean()
                # total_val_acc += accuracy
                # total_val_acc = accuracy_score(target,metrics)
                #r2_score를 위한
                true = torch.Tensor(target).cuda()
                preds = torch.cat((preds, metrics))
                trues = torch.cat((trues, true))
                    
            r2_res = r2_score(trues.cpu().numpy().squeeze(), preds.cpu().numpy().squeeze(), multioutput='raw_values')
            total_val_loss /= len(dataloader_te) 
            # total_val_acc /= len(dataloader_te)
            
            print(f'[{epoch+1}epoch] tr_loss:{total_loss:.4f} | val_loss:{total_val_loss:.4f} | R2_Score:{r2_res}') 
        

            


[1epoch] tr_loss:0.2855 | val_loss:0.2815 | R2_Score:[-47.28687282  -0.16777562]
[11epoch] tr_loss:0.1832 | val_loss:0.1813 | R2_Score:[-29.66356738  -0.04159952]
[21epoch] tr_loss:0.0934 | val_loss:0.0935 | R2_Score:[-14.2158887    0.05283572]
[31epoch] tr_loss:0.0409 | val_loss:0.0410 | R2_Score:[-5.01806931  0.13249039]
[41epoch] tr_loss:0.0230 | val_loss:0.0236 | R2_Score:[-2.01794002  0.21392161]
[51epoch] tr_loss:0.0180 | val_loss:0.0185 | R2_Score:[-1.21516912  0.28311025]
[61epoch] tr_loss:0.0158 | val_loss:0.0164 | R2_Score:[-0.92964998  0.33457579]
[71epoch] tr_loss:0.0152 | val_loss:0.0151 | R2_Score:[-0.76442737  0.37426559]
[81epoch] tr_loss:0.0139 | val_loss:0.0141 | R2_Score:[-0.63396984  0.40552327]
[91epoch] tr_loss:0.0129 | val_loss:0.0133 | R2_Score:[-0.52357493  0.43312833]
[101epoch] tr_loss:0.0118 | val_loss:0.0125 | R2_Score:[-0.4185635   0.45785093]
[111epoch] tr_loss:0.0112 | val_loss:0.0117 | R2_Score:[-0.32219519  0.48057582]
[121epoch] tr_loss:0.0106 | val_l